In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
#from pmdarima.arima import auto_arima
#from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import numpy as np

In [ ]:
data = pd.read_csv("/kaggle/input/for-simple-exercises-time-series-forecasting/Alcohol_Sales.csv")

data=data.sort_values(by='DATE')

data.head()
data.columns=['DATE', 'SALES']
data.head()

In [ ]:
data['DATE'] = pd.to_datetime(data['DATE'], infer_datetime_format = True)
data = data.set_index('DATE')
data.head()

In [ ]:
data.shape

In [ ]:
#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Close Price')
plt.plot( data['SALES'])
plt.title('closing price')
plt.show()

In [ ]:
df_alc = data['SALES']
df_alc.plot(style='k.')
plt.title('Scatter plot of closing price')
plt.show()

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    
    print("Augmented Dicky-Fuller Test")
    labels=['Adf Test Statistics', 'p-value', '# of lags', 'Num of Observations used']
    
    for value,label in zip(adft,labels):
        print(label+ ":"+str(value))
        
    if adft[1]<= 0.05:
        print("Strong evidence against null hypothesis")
        print("reject null hypotesis")
        print("data has no unit root and is stationary")
    else:
        print('weak evidence against null hypothesis')
        print('Fail to reject null hypo')
        print('Data has a unit root, it is a non-stationary')
    
    
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
        
        
    
test_stationarity(df_alc)

In [ ]:
result = seasonal_decompose(df_alc, model='multiplicative', freq = 30)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
#df_log = np.log(df_close)
df_log = data['SALES'] - data['SALES'].shift(1)
df_log=df_log.dropna(axis=0)
moving_avg = df_log.rolling(12).mean()
std_dev = df_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
test_stationarity(df_log)

In [ ]:
#split data into train and training set
train_data= df_log.iloc[:(int(len(df_log)*0.9))]
test_data =df_log.iloc[int(len(df_log)*0.9):len(df_log)-1]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(df_log, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

Using Auto_Arima to find p q d

In [ ]:
!pip install pmdarima

unfortunately I could not install pmdarima successfully. but I used uato_arima to find the proper p,q,d and used it in ARIMA Model

In [ ]:
from pmdarima import auto_arima
# help(auto_arima) Get info on parameters

# Turn off warnings that can be distracting and of no concern
import warnings
warnings.filterwarnings('ignore')


# Trace shows the 1st couple ARIMA models it tried to fit
step_fit = auto_arima(train_data, m=12, trace=True)

# Returns AIC and BIC which estimate the quality of the models
# Then summary provides the best performing model
step_fit.summary()

# the Results:

In [ ]:
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,1,1)[12] intercept   : AIC=3991.559, Time=3.66 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=4316.663, Time=0.03 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=4196.208, Time=0.68 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=4101.995, Time=1.70 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=4314.669, Time=0.05 sec
 ARIMA(2,0,2)(0,1,1)[12] intercept   : AIC=4004.520, Time=2.96 sec
 ARIMA(2,0,2)(1,1,0)[12] intercept   : AIC=4012.965, Time=1.53 sec
 ARIMA(2,0,2)(2,1,1)[12] intercept   : AIC=3978.383, Time=7.74 sec
 ARIMA(2,0,2)(2,1,0)[12] intercept   : AIC=3999.018, Time=4.15 sec
 ARIMA(2,0,2)(2,1,2)[12] intercept   : AIC=3940.966, Time=10.57 sec
 ARIMA(2,0,2)(1,1,2)[12] intercept   : AIC=3985.943, Time=8.40 sec
 ARIMA(1,0,2)(2,1,2)[12] intercept   : AIC=4007.469, Time=9.24 sec
 ARIMA(2,0,1)(2,1,2)[12] intercept   : AIC=3946.010, Time=8.16 sec
 ARIMA(3,0,2)(2,1,2)[12] intercept   : AIC=3945.498, Time=10.87 sec
 ARIMA(2,0,3)(2,1,2)[12] intercept   : AIC=inf, Time=12.22 sec
 ARIMA(1,0,1)(2,1,2)[12] intercept   : AIC=inf, Time=8.24 sec
 ARIMA(1,0,3)(2,1,2)[12] intercept   : AIC=3949.710, Time=10.47 sec
 ARIMA(3,0,1)(2,1,2)[12] intercept   : AIC=3949.616, Time=11.37 sec
 ARIMA(3,0,3)(2,1,2)[12] intercept   : AIC=3938.099, Time=12.81 sec
 ARIMA(3,0,3)(1,1,2)[12] intercept   : AIC=inf, Time=11.52 sec
 ARIMA(3,0,3)(2,1,1)[12] intercept   : AIC=inf, Time=15.69 sec
 ARIMA(3,0,3)(1,1,1)[12] intercept   : AIC=inf, Time=5.59 sec
 ARIMA(4,0,3)(2,1,2)[12] intercept   : AIC=inf, Time=14.75 sec
 ARIMA(3,0,4)(2,1,2)[12] intercept   : AIC=3935.962, Time=17.09 sec
 ARIMA(3,0,4)(1,1,2)[12] intercept   : AIC=3951.287, Time=16.07 sec
 ARIMA(3,0,4)(2,1,1)[12] intercept   : AIC=inf, Time=14.92 sec
 ARIMA(3,0,4)(1,1,1)[12] intercept   : AIC=inf, Time=6.71 sec
 ARIMA(2,0,4)(2,1,2)[12] intercept   : AIC=inf, Time=13.15 sec
 ARIMA(4,0,4)(2,1,2)[12] intercept   : AIC=3934.063, Time=15.91 sec
 ARIMA(4,0,4)(1,1,2)[12] intercept   : AIC=3945.000, Time=15.61 sec
 ARIMA(4,0,4)(2,1,1)[12] intercept   : AIC=3944.129, Time=13.43 sec
 ARIMA(4,0,4)(1,1,1)[12] intercept   : AIC=3944.136, Time=6.89 sec
 ARIMA(5,0,4)(2,1,2)[12] intercept   : AIC=inf, Time=18.75 sec
 ARIMA(4,0,5)(2,1,2)[12] intercept   : AIC=inf, Time=18.35 sec
 ARIMA(3,0,5)(2,1,2)[12] intercept   : AIC=inf, Time=17.30 sec
 ARIMA(5,0,3)(2,1,2)[12] intercept   : AIC=3939.469, Time=17.33 sec
 ARIMA(5,0,5)(2,1,2)[12] intercept   : AIC=inf, Time=18.60 sec
 ARIMA(4,0,4)(2,1,2)[12]             : AIC=inf, Time=19.03 sec

Best model:  ARIMA(4,0,4)(2,1,2)[12] intercept
Total fit time: 401.847 seconds

Using plot_diagnostics we can plot the results of auto_arima

In [ ]:
step_fit.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(train_data, order=(4, 0, 4))  
fitted = model.fit(disp=-1)  
print(fitted.summary())# Forecast
fc, se, conf = fitted.forecast(32, alpha=0.05)  # 95% confidence
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data.index)
upper_series = pd.Series(conf[:, 1], index=test_data.index)
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data, label='training')
plt.plot(test_data, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('Altaba Inc. Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))

In [ ]:
pd.DataFrame({'test':test_data,'pred':fc_series}).plot()